# Modelo predictivo

### Importar paquetes

In [82]:
import pandas as pd
import os
import re

### Limpieza de los datos

Empezaremos por todas las transformaciones necesarias tras el análisis previo. El objetivo es conseguir un conjunto de entrenamiento y test para nuestros modelos.

A pesar de que el reto específica que se nos proporcionarían diferentes conjuntos de test, tan solo contamos con uno. Eso nos hizo sospechar, tras un examen manual, nos dimos cuenta que los lotes de test están mezclados con el resto. Así que lo primero será separar dichos lotes y a partir de ahi construir la tabla con los campos relevantes. 

Este proceso será iterativo, por lo que complica la entrega, ya que solo podemos entregar un archivo. Por tanto se irá documentando, las diferentes transformaciones que se harán en distintos puntos del proceso hasta conseguir un modelo con buen rendimiento.

Como primer acercamiento será obtener los valores agregados de las distintas fases para poder predecir la variable objetivo.
Para ello necesitaremos crear un `pipeline` que garantize el orden de los pasos, dado que la entrada que se nos proporciona para el test es insuficiente para realizar una predicción

In [2]:
df_cultivo_test = pd.read_excel('raw_data/Fases producción_test v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo_test.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,24054,1,NaN,14616,2024-04-16 08:12:00,2024-04-18 07:28:00,81.6,15.44,85.60,184800000,14246.0,27.84,23.96,NaN,NaN
1,24055,1,NaN,14614,2024-04-13 08:18:00,2024-04-15 08:30:00,NaN,14.32,73.68,175200000,12912.0,30.96,23.16,NaN,NaN
2,24056,1,NaN,14615,2024-04-13 08:18:00,2024-04-15 08:15:00,NaN,14.56,82.40,168000000,14246.0,29.52,28.88,NaN,NaN
3,24057,1,NaN,13170,2024-04-16 08:12:00,2024-04-18 07:41:00,82.4,17.76,78.96,180800000,12912.0,31.04,25.32,NaN,NaN
4,24058,2,24055.0,14614,2024-04-15 12:28:00,2024-04-17 08:14:00,87.2,18.00,82.40,144800000,12912.0,26.08,20.36,NaN,NaN


In [3]:
lotes_test = df_cultivo_test['LOTE'].unique()
len(lotes_test)

56

In [4]:
df_cultivo = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,23019,1,NaN,14615,2023-03-21 07:30:00,2023-03-23 06:30:00,82.4,17.28,91.2,184000000,17825,NaN,NaN,1747.920,6.00
1,23020,1,NaN,14616,2023-03-21 07:30:00,2023-03-23 06:30:00,80.4,18.80,91.2,181600000,14246,NaN,NaN,1676.160,6.56
2,23021,1,NaN,13170,2023-03-22 07:30:00,2023-03-24 06:30:00,66.4,16.16,86.4,248000000,17825,NaN,NaN,1928.496,8.08
3,23022,1,NaN,14614,2023-03-22 07:30:00,2023-03-24 06:30:00,85.6,18.48,83.2,229600000,12912,NaN,NaN,1782.800,5.92
4,23023,1,NaN,14615,2023-03-28 07:27:00,2023-03-30 10:00:00,77.6,17.12,74.4,132800000,17825,26.56,20.88,1861.840,2.96


___

In [55]:
def get_orden_data():
    # Cargamos datos
    df = pd.read_excel('raw_data/OF 123456 v02.xlsx', engine='openpyxl')

    # Corregimos caracteres especiales en Lote 
    df['LOTE'] = df['Lote'].apply(lambda x: int(x.replace('/', '').replace('P', '')))

    # Renombrado de columnas
    df = df.rename(columns={'Orden': 'orden', 'Cantidad entregada': 'cantidad'})
    
    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['orden', 'cantidad']

    return df[final_cols]

In [56]:
def get_preinoculo_data(lotes):
    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Preinóculo', header=[0, 1], na_values=['NA', 'N.A'], engine='openpyxl')
    # Corregir cabeceras
    new_cols = [l2 if 'Unnamed' in l1 else f"{l1}-{l2}" for l1, l2 in df.columns.to_list()]
    df.columns = new_cols

    # Filtro de lotes
    df = df[df['LOTE'].isin(lotes)]

    # Imputación valores perdidos por la media
    for c in df.select_dtypes(include='float').columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(df[c].mean())
        
    # Valores de pH y turbidez seleccionados para siguiente fase
    ph_cols = [col for col in df.columns if col.startswith('pH')]
    turbidez_cols = [col for col in df.columns if col.startswith('Turbidez')]
    lineas_cols = [col for col in df.columns if col.startswith('Línea')]

    # el ph y turbidez finales las consideramos como la mezcla de las dos líneas escogidas
    df['ph'] = (
        (
            df[ph_cols].values *
            df[lineas_cols].values
        ).sum(axis=1) /
        df[lineas_cols].sum(axis=1)
    ).round(3)
    
    df['turbidez'] = (
        (
            df[turbidez_cols].values *
            df[lineas_cols].values
        ).sum(axis=1) /
        df[lineas_cols].sum(axis=1)
    ).round(2)

    # Calculo duración
    # 1. obtenemos las horas
    # 2. corregimos las fechas "al reves"
    # 3. corregimos las horas negativas
    ini_col = 'Fecha/hora inicio'
    fin_col = 'Fecha/hora fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['ph','turbidez','duracion_horas']

    return df[final_cols]


In [57]:
def get_inoculo_data(lotes, df_preinoculo):
    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Inóculo', engine='openpyxl')
    df['Viabilidad final cultivo'] = pd.to_numeric(df['Viabilidad final cultivo'], errors='coerce')

    # Filtrado de lotes 
    df = df[df['LOTE'].isin(lotes)]
    df = df.dropna(subset=['Fecha/hora inicio','Fecha/hora fin'])

    # Renombrado de columnas
    df = df.rename(columns={'Fecha/hora inicio': 'ts_inicio', 
                            'Fecha/hora fin': 'ts_fin', 
                            'ID bioreactor': 'id_bioreactor',
                            'Turbidez inicio cultivo': 'turbidez_ini',
                            'Turbidez final culttivo': 'turbidez_fin',
                            'Viabilidad final cultivo': 'viabilidad'}
    )

    # Imputación de valores perdidos
    df.loc[df['turbidez_ini'].isna(), ['turbidez_ini']] = (
        df[df['turbidez_ini'].isna()]
        .join(df_preinoculo, 
              on='LOTE', how='left', rsuffix='preinoculo')
        ['turbidez']
    )
    df['turbidez_dif'] = df['turbidez_fin'] - df['turbidez_ini']

    # > Si con la anterior imputación volvemos a tener un NaN en la resta, imputamos por la media
    df['turbidez_dif'] = df['turbidez_dif'].fillna(df['turbidez_dif'].mean())
    
    # Calculo duracion de la fase en horas
    ini_col = 'ts_inicio'
    fin_col = 'ts_fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['id_bioreactor', 'ts_inicio', 'ts_fin', 'turbidez_dif', 'viabilidad', 'duracion_horas']

    return df[final_cols]

In [70]:
def get_cultivo_data(lotes):

    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
    df['Viabilidad final cultivo'] = pd.to_numeric(df['Viabilidad final cultivo'], errors='coerce')

    # Filtrado de lotes 
    df = df[df['LOTE'].isin(lotes)]

    # Renombrado de columnas
    df = df.rename(columns={'Fecha/hora inicio': 'ts_inicio', 
                            'Fecha/hora fin': 'ts_fin', 
                            'ID Bioreactor': 'id_bioreactor',
                            'ID Centrífuga': 'id_centrifugadora',
                            'LOTE parental': 'lote_padre',
                            'Volumen de inóculo utilizado': 'volumen_ini',
                            'Turbidez inicio cultivo': 'turbidez_ini',
                            'Turbidez fin cultivo': 'turbidez_fin',
                            'Centrifugación 1 turbidez':'turbidez_cfg1',
                            'Centrifugación 2 turbidez':'turbidez_cfg2',
                            'Viabilidad final cultivo': 'viabilidad',
                            'Producto 1': 'producto'}
    )

    # Imputar valores perdidos
    df['lote_padre'] = df['lote_padre'].fillna(0)
    df['lote_padre'] = df['lote_padre'].astype(int)
    # La variable volumen del dataset de inoculo no tiene misma semantica, se imputa por la media (no hay demasiada desviacion)
    df['volumen_ini'] = df['volumen_ini'].fillna(df['volumen_ini'].mean())

    # NO ME GUSTA ESTA IDEA
    df['turbidez_cfg1'] = df['turbidez_cfg1'].fillna(df['turbidez_cfg1'].mean())
    df['turbidez_cfg2'] = df['turbidez_cfg2'].fillna(df['turbidez_cfg2'].mean())

    # Calculos de diferencias
    df['turbidez_dif'] = (df['turbidez_fin'] - df['turbidez_ini']).round(2) # la turbidez aumenta en los bioreactores
    df['turbidez_cfg_dif'] = (df['turbidez_cfg1'] - df['turbidez_cfg2']).round(2)  # la turbidez disminuye tras cada centrifugado


    # Calculo duracion de la fase en horas
    ini_col = 'ts_inicio'
    fin_col = 'ts_fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['id_bioreactor', 'id_centrifugadora', 'lote_padre', 'ts_inicio', 'ts_fin', 
                  'volumen_ini', 'turbidez_dif', 'turbidez_cfg_dif', 'duracion_horas', 'producto']

    return df[final_cols]
    

In [107]:
def concatenate_bioreactor_data():
    bioreactores_files = [f for f in os.listdir('./raw_data') if not os.path.isdir(f) and f.startswith('Biorreactor')]
    bioreactores_dfs = []
    for bio_fname in bioreactores_files:
        
        numbers = re.findall(r'\d+', 'Biorreactor 14618.xlsx')
        id_bioreactor = int(numbers[0])

        df = pd.read_excel(f'./raw_data/{bio_fname}', sheet_name='Datos', index_col='DateTime', parse_dates=True, engine='openpyxl')
        
        clean_cols = [col.split('.')[1].lower() for col in df.columns]
        renamed_cols = {old: new for old, new in zip(df.columns, clean_cols)}
        
        df = df.rename(columns=renamed_cols)
        df['id_bioreactor'] = id_bioreactor
        bioreactores_dfs.append(df)

    df = pd.concat(bioreactores_dfs, axis=0).sort_index().set_index('id_bioreactor', append=True)
    return df

In [108]:
concatenate_bioreactor_data() # revisar los indices !!

agitation_pv  air_sparge_pv  \
DateTime   id_bioreactor                                
2023-03-15 14618                  72.0       0.000000   
           14618                  80.0       0.000000   
           14618                   0.0       0.000000   
           14618                  80.0       0.000000   
           14618                  80.0       0.000000   
...                                ...            ...   
2024-09-11 14618                   0.0       0.000000   
           14618                   0.0       0.000000   
           14618                  20.0       0.000000   
           14618                  80.0       0.000000   
           14618                  80.0      64.313885   

                          biocontainer_pressure_pv    do_1_pv    do_2_pv  \
DateTime   id_bioreactor                                                   
2023-03-15 14618                          1.869612  15.953993        NaN   
           14618                          0.572660   0.000000  -0.005530   
           14618                        480.000000   0.000000        NaN   
           14618                          0.268969  18.732030   0.000000   
           14618                          0.715311  16.557993   0.000000   
...                                            ...        ...        ...   
2024-09-11 14618                         -0.579450   0.000000   0.000000   
           14618                        480.000000   0.000000   0.000000   
           14618                         -2.052007   0.000000   0.000000   
           14618                         -0.399304   0.000000  16.229234   
           14618                          2.397993  14.085248   0.000000   

                          gas_overlay_pv  load_cell_net_pv     ph_1_pv  \
DateTime   id_bioreactor                                                 
2023-03-15 14618                4.000057            169.60 -527.387158   
           14618                4.000087           1576.80   -0.156925   
           14618                0.000000              0.00    1.707277   
           14618                3.999874           1636.80    5.914182   
           14618                3.999639           1652.80    5.929625   
...                                  ...               ...         ...   
2024-09-11 14618                4.000006             -1.44    1.444136   
           14618                0.000000            -17.60    1.415245   
           14618                3.999893            161.60    5.072766   
           14618                3.999885           1660.00    5.874518   
           14618                4.003011           1664.00    5.878986   

                             ph_2_pv  pump_1_pv  pump_1_total  pump_2_pv  \
DateTime   id_bioreactor                                                   
2023-03-15 14618            5.998478        0.0    104.159973   0.000000   
           14618            5.888288        0.0     14.880000   0.000000   
           14618           -1.969051        0.0     49.599991   0.000000   
           14618           -0.234763        0.0     22.439996   0.000000   
           14618         -389.260962        0.0     39.679996   0.000000   
...                              ...        ...           ...        ...   
2024-09-11 14618           -0.613304        0.0      0.000000   0.000000   
           14618           -0.172975        0.0     59.519989   0.000000   
           14618           -0.016351        0.0      7.439999   0.000000   
           14618           -0.347235        0.0     29.760001   7.182158   
           14618           -0.164859        0.0     32.400000   0.971887   

                          pump_2_total  single_use_do_pv  single_use_ph_pv  \
DateTime   id_bioreactor                                                     
2023-03-15 14618              0.000000         16.713527          6.016000   
           14618            191.200293        799.991992        799.967969   
           14618           2014

In [71]:
df_info_general = get_orden_data()
lotes = df_info_general.index.unique()

df_preinoculo = get_preinoculo_data(lotes)
df_inoculo = get_inoculo_data(lotes, df_preinoculo)

df_cultivo = get_cultivo_data(lotes)

In [72]:
df_cultivo

,id_bioreactor,id_centrifugadora,lote_padre,ts_inicio,ts_fin,volumen_ini,turbidez_dif,turbidez_cfg_dif,duracion_horas,producto
LOTE,,,,,,,,,,
23019,14615,17825,0,2023-03-21 07:30:00,2023-03-23 06:30:00,82.40,73.92,6.44,47.00,1747.920
23020,14616,14246,0,2023-03-21 07:30:00,2023-03-23 06:30:00,80.40,72.40,6.44,47.00,1676.160
23021,13170,17825,0,2023-03-22 07:30:00,2023-03-24 06:30:00,66.40,70.24,6.44,47.00,1928.496
23022,14614,12912,0,2023-03-22 07:30:00,2023-03-24 06:30:00,85.60,64.72,6.44,47.00,1782.800
23023,14615,17825,0,2023-03-28 07:27:00,2023-03-30 10:00:00,77.60,57.28,5.68,50.55,1861.840
...,...,...,...,...,...,...,...,...,...,...
24049,14617,12912,0,2024-03-16 09:22:00,2024-03-18 08:23:00,83.60,53.76,13.56,47.02,1342.800
24050,14614,6379,0,2024-03-23 08:57:00,2024-03-25 08:28:00,84.16,49.84,2.80,47.52,1422.800
24051,13169,12912,0,2024-03-23 08:57:00,2024-03-25 08:33:00,84.16,63.04,14.12,47.60,1486.560
